# Team 5 - Learning Models by Exploiting Structure and Fundamental Knowledge

# SOLUTIONS NOTEBOOK


1. [Introduction](#robot-world)
    1. [Computing Gradient of a Function - Mini Tutorial](#gradient-tutorial)
    2. [Loss Functions](#loss-functions)
    3. [Worked Example](#example)
2. [Problem A: Temporal Cohesion Prior (15 points)](#partA)
3. [Problem B: Causality Prior (15 points)](#partB)
4. [Problem C: Repeatability Prior (20 points)](#partC)

# Introduction <a id='robot-world'/>

We have deployed our robot in a simple 2D world, and now the robot needs our help to learn the state representation of its world. The world is a 10x10 grid with our robot occupying one of the cells. The robot knows what actions it can take in this envrionment and also gets rewards. The robot gets more rewards the further away it travels from (0,0) on the grid.

As students of Cognitive Robotics course, the robot has come to you to help improve its learning model so that he can have an accurate state representation of where it is in the world. 

## Computing Gradient of a Function - Mini Tutorial <a id='gradient-tutorial'/>

The robot has found some material online to help you refresh your memory on how to take gradient of a function.
Let's take a look at some basic simple examples:

$f(x) = x^{4} + x + 1$. What is the $\nabla f(x)$ at $x = 1$ ?

We take the partial derivative with respect to $x$, $f_{x}$ as

$f_{x} = 4 * x^{3} + 1$

Then, $\nabla f(x = 1) = 4 * (1)^{3} + 1 = 5$

$f(x,y) = e^{2x} \sin(3y)$. What is the $\nabla f(x,y)$ at $x = 2, y = -2$ ?

We can formulate the gradient of the function $\nabla f(x,y) = <f_{x}, f_{y}>$ where $f_{x}$ and $f_{y}$ are partial derivates with respect to $x$ and $y$ respectively.

$f_{x} = 2 * e^{2x} \sin(3y)$

$f_{y} = 3 * e^{2x} \cos(3y)$

Finally, $\nabla f(2,-2) = <2 * e^{2x} \sin(3y),  3 * e^{2x} \cos(3y)>$

$\nabla f(2,-2) = <2 * e^{4} \sin(-6),  3 * e^{4} \cos(-6)>$

$\nabla f(2,-2) = <37.51,  157.27>$

Here is an [online tutorial](https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/partial-derivative-and-gradient-articles/a/introduction-to-partial-derivatives) for revising partial derivatives.

In [1]:
import checker
import numpy as np
from utils_pset import Data_Frame

## Loss Functions <a id='loss-functions'/>

In lecture, we stepped through multiple components of a loss function that is used to learn physics-based state representations. However, we do not use the loss function directly during learning, but rather the gradient of the loss function to perform gradient descent. In this problem, you will implement the gradient of each component of the loss function.

The incoming data will be formatted as an array of Data_Frame objects which are defined in utils_pset. At a high level, these objects have:

Using the incoming data as well as the matrix used to transform image vectors to coordinates, you will step through and implement the loss function defined in lecture that biases state representations toward following Newton's laws of physics. Each section will step you through implementing a different component of the loss function: temporal cohesion, proportionality, causality, repeatability. These are the physical prior of the environment for this representation.

The mapping matrix is the matrix we are learning to transform high dimensional image vectors to low dimensional state representations. Wherever we use the variable $s_t$ to represent a state, this can be thought of as $s_t = m v_t$, where $m$ is the mapping matrix that we are learning, and $v_t$ is the image vector taken of the world at time $t$.


## API

Data will be given to you as an array of Data_Frame objects with the following attributes:

* `frame.action`: an integer indicating the action taken at the given timestep (up, down, left right)
*  `frame.reward`: the reward received at the given timestep for moving away from a position.
*  `frame.image`: a flattened array of the pixel values of the image taken of the agent (the robot on the grid)

Additionally, the mapping matrix will be passed as input as an 2x100 matrix, since our state representation
dimension is 2, and our image vector dimension is 100 (from the 10x10 image).

Remember: you can use `np.linalg.norm` to take the magnitude of a vector.


## Worked Example <a id='example'/>

First, we want our agent to learn state representations that follow the rule that F=ma. This means we need to implement the Proportionality Prior Gradient.

The proportionality prior loss is defined as $E[(||\triangle s_{t_2}|| - ||\triangle s_{t_1}||)^2 | a_{t_1} = a_{t_2}]$, for two time steps $t_1$ and $t_2$. 

We have implemented this for you to give you an example of how to implement the remaining loss function components. We have broken the gradient into two components — an outer loop and an innter loop. The outer loop (executed in `proportionality_prior_gradient`) corresponds to the expectation portion of the function, where we want to find time steps where the agent took the same action. 

We then use a helper function, `proportional_loss_gradient`, to compute the gradient of $||\triangle s_{t_2}|| - ||\triangle s_{t_1}||)^2$. Now remember, to compute this gradient, it is useful to think of the function less in terms of the state representation themselves $s$, but rather in terms of the image vectors ($v_{t_1}, v_{t_1+1}, v_{t_2}, v_{t_2+1}$) and the mapping matrix ($m$). Which would give you:

$\frac{d}{dm} (||m v_{t_1 +1} - m v_{t_1}|| - ||m v_{t_2 +1} - m v_{t_2}||)^2$

Which is important to think about when computing gradients because your gradients are always with respect to $m$, which is concealed inside the state representations. This derivative is computer below by iterating over each element of $m$ and computing the gradient of it with respect to the rest of the lost function, and then we return the entire gradient of $m$. 

You can see the gradient computation is grounded, not in the individual value of the state representations, but rather in the delta between state representations, and is focused on measuring the difference between those deltas. In implementing the gradients for the remaining functions, we hope you can refine your mathematical intuition behind the formulation of these loss function components 

In [2]:
def proportionality_prior_gradient(batch, mapping):
    """
    :param batch an array of Data_Frame objects
    :param mapping the current matrix that maps flattened image vectors to coordinates
    :return the gradient of the proportionality prior loss (should be the same dimensions as the mapping matrix)
    """

    time_steps = len(batch)
    total_loss_grad = np.zeros(mapping.shape)
    pairings = 0
    for i in range(0, time_steps - 1):
      # for each time step
      for j in range(i+1,time_steps - 1):
          # look at each later time step (to avoid double counting)
          if batch[i].action == batch[j].action:
              # if we have the same action, calculate the gradient for this
              # time step pairing and add to the total
              pairings += 1
              loss_grad = proportional_loss_gradient(batch[i].image, batch[i + 1].image, batch[j].image, batch[j+1].image, mapping)
              total_loss_grad += loss_grad.reshape(mapping.shape)
    # return the average over all of the pairings we found
    return total_loss_grad / pairings


def proportional_loss_gradient(s1, s2, s3, s4, mapping):
    """
    :param s1 image vector at t1
    :param s2 image vector at t1+1
    :param s3 image vector at t2
    :param s4 image vector at t2+1
    :return the loss gradient for just the t1/t2 time step pairing
    """

    dims = mapping.shape
    output = np.zeros(dims)

    delta1 = mapping@s2 - mapping@s1
    delta2 = mapping@s4 - mapping@s3

    #first layer of chain rule from derivative calculation
    outest = (np.linalg.norm(delta2) - np.linalg.norm(delta1))*2

    # iterate over each element of the mapping
    for i in range(0, dims[0]):
        # second later of chain rule
        outer_denom_1 = delta1[i]*np.linalg.norm(delta1)
        outer_denom_2 = delta2[i]*np.linalg.norm(delta2)
        for j in range(0, dims[1]):
            # final layer of chain rule
            frac_1 = (delta1[i]**2)*(s1[j]- s2[j])/outer_denom_1
            frac_2 = (delta2[i]**2)*(s3[j] - s4[j])/outer_denom_2
            output[i,j] = -(frac_2 - frac_1)*outest

    return output

Here is a toy example to demonstrate what the output of this proportionality prior may look. Instead of using
a 10x10 image though, we will use a 2x2 image to make the answer easier to parse.

In [3]:
image1 = np.array([0, 1, 1, 1]).T  # (0,0)
image2 = np.array([1, 0, 1, 1]).T  # (1,0)
image3 = np.array([1, 1, 0, 1]).T  # (0,0)
image4 = np.array([1, 1, 1, 0.1]).T  # (1,0)
frame1 = Data_Frame(np.array([1, 2]), 4, 1, image1)
frame2 = Data_Frame(np.array([2, 3]), 4, 2, image2)
frame3 = Data_Frame(np.array([2, 1]), 4, 1, image3)
frame4 = Data_Frame(np.array([2, 4]), 4, 2, image4)
mapping = np.arange(8).reshape((2, 4))

batch = [frame1, frame2, frame3, frame4]

for f in batch:
    print("frame image of coords: {}\n{}".format(f.coords, f.image))
    print(f.image.reshape((2,2)))

print("mapping: {}".format(mapping))
print(proportionality_prior_gradient(batch, mapping))

frame image of coords: [1 2]
[0 1 1 1]
[[0 1]
 [1 1]]
frame image of coords: [2 3]
[1 0 1 1]
[[1 0]
 [1 1]]
frame image of coords: [2 1]
[1 1 0 1]
[[1 1]
 [0 1]]
frame image of coords: [2 4]
[1.  1.  1.  0.1]
[[1.  1. ]
 [1.  0.1]]
mapping: [[0 1 2 3]
 [4 5 6 7]]
[[-0.30765568  0.          1.10747884 -0.71984084]
 [-0.30765568  0.          0.65043703 -0.30850322]]


## Problem A: Temporal Cohesion Prior (15 points) <a id='partA'/>

The temporal cohesion loss represents Newton's Law of Inertia by penalizing temporally adjacent timeframes for being too different from each other.

In lecture, we defined the temporal cohesion loss as $E[||\triangle s_t||^2]$, where $s_t$ is the state representation at time $t$.
Use the function skeleton below to implement the gradient of the temporal cohesion loss.

Hint: You will want to take the derivative of $E[||\triangle s_t||^2]$ with respect to the `mapping` matrix, and implement that as part of the code.

In [4]:
def temporal_cohesion_gradient(batch, mapping):
    """
    :param batch an array of Data_Frame objects
    :param mapping the current matrix that maps flattened image vectors to coordinates
    :return the gradient of the temporal cohesion loss (should be the same dimensions as the mapping matrix)
    """

    ### BEGIN SOLUTION
    time_steps = len(batch)
    total_loss_grad = np.zeros(mapping.shape)
    for i in range(0,time_steps-1):
        loss_grad = temporal_loss_gradient(batch[i], batch[i+1], mapping)
        total_loss_grad += loss_grad.reshape(mapping.shape)

    return total_loss_grad/(time_steps-1)

def temporal_loss_gradient(init_state, next_state, mapping):
    dims = mapping.shape
    output = np.zeros(dims)
    for i in range(0, dims[0]):
        outside_comp = mapping[i,:]@init_state.image - mapping[i,:]@next_state.image
        for j in range(0, dims[1]):
            output[i,j] = 2*outside_comp*(init_state.image[j] - next_state.image[j])
    return output
    
    ### END SOLUTION

In [5]:
checker.temporal_sol_check(temporal_cohesion_gradient)
checker.test_ok()

Tests passed!!

## Problem B: Causality Prior (15 points) <a id='partB'/>

The causality prior loss represents Newton's Third Law of equal and opposite reactions by penalizing same actions with different rewards having similar state representations.

In lecture, we defined the proportionality prior loss between two timesteps $t_1$ and $t_2$ as $E[e^{-|| s_{t_2} -  s_{t_1}||} | a_{t_1} = a_{t_2} \wedge r_{t_1} \neq r_{t_2}]$.
Use the function skeleton below to implement the gradient of the causality prior loss.

In [6]:
def causality_prior_gradient(batch, mapping):
    """
    :param batch an array of Data_Frame objects
    :param mapping the current matrix that maps flattened image vectors to coordinates
    :return the gradient of the causality prior loss (should be the same dimensions as the mapping matrix)
    """

    ### BEGIN SOLUTION
    time_steps = len(batch)
    total_loss_grad = np.zeros(mapping.shape)
    pairings = 0
    for i in range(0, time_steps - 1):

        for j in range(i + 1, time_steps - 1):
            pairings += 1
            if batch[i].action == batch[j].action and batch[i].reward != batch[j].reward:
                loss_grad = causal_loss_gradient(batch[i].image, batch[j].image,
                                                mapping)
                total_loss_grad += loss_grad.reshape(mapping.shape)

    return total_loss_grad / pairings

def causal_loss_gradient(s1,s2,mapping):
    dims = mapping.shape
    output = np.zeros(dims)

    delta = mapping@s1 - mapping@s2
    delta_norm = np.linalg.norm(delta)
    for i in range(0,dims[0]):
        denom = 2*delta[i]*delta_norm
        for j in range(0,dims[1]):
            numer = delta[i]*np.exp(-delta_norm) * (s1[j] - s2[j])*delta[i]*2
            output[i,j] = -numer/denom
    return output

    ### END SOLUTION

In [7]:
checker.causal_sol_check(causality_prior_gradient)
checker.test_ok()

Tests passed!!

## Problem C: Repeatability Prior (15 points) <a id='partC'/>

The repeatability prior loss represents Newton's Third Law of equal and opposite reactions by penalizing same actions taken from similar positions for transitioning to different positions.

In lecture, we defined the proportionality prior loss between two timesteps $t_1$ and $t_2$ as $E[e^{-|| s_{t_2} -  s_{t_1}||}||\triangle s_{t_2} - \triangle s_{t_1}||^2  | a_{t_1} = a_{t_2}]$.
Use the function skeleton below to implement the gradient of the repeatability prior loss.

In [8]:
def repeatability_prior_gradient(batch, mapping):
    """
    :param batch an array of Data_Frame objects
    :param mapping the current matrix that maps flattened image vectors to coordinates
    :return the gradient of the repeatability prior loss (should be the same dimensions as the mapping matrix)
    """

    ### BEGIN SOLUTION
    time_steps = len(batch)
    total_loss_grad = np.zeros(mapping.shape)
    pairings = 0
    for i in range(0, time_steps - 1):
        for j in range(i + 1, time_steps - 1):
            if batch[i].action == batch[j].action:
                pairings += 1
                loss_grad = proportional_loss_gradient(batch[i].image, batch[i + 1].image, batch[j].image,
                                                       batch[j + 1].image, mapping)
                total_loss_grad += loss_grad.reshape(mapping.shape)

    return total_loss_grad / pairings


def repeatability_loss_gradient(s1, s2, s3, s4, mapping):
    """
    :param batch:
    :return:
    """
    dims = mapping.shape
    output = np.zeros(dims)

    delta1 = (mapping@s2 - mapping@s1)
    delta2 = (mapping@s4 - mapping@s3)
    base_priorish_loss = np.linalg.norm(delta2 - delta1)**2
    base_causal_loss = np.exp(-np.linalg.norm(mapping@s3 - mapping@s1))
    causal_losses = causal_loss_gradient(s1, s3, mapping)

    for i in range(0,dims[0]):
        outer = delta2[i] - delta1[i]
        for j in range(0,dims[1]):
            inner = s4[j] - s3[j] - s2[j] + s1[j]
            output[i,j] = 2*inner*outer*base_causal_loss + causal_losses[i,j]*base_priorish_loss
    return output
    ### END SOLUTION

In [9]:
checker.repeatability_sol_check(repeatability_prior_gradient)
checker.test_ok()

Tests passed!!

## DONE!! Great Work!